# Smart Feed Recommender System

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout,Input, Dot

# Text Preprocessing

Consider required columns\
Missing Data\
Duplicated Data\
Keywords\
Stop words\
PorterStemmer


In [319]:
df = pd.read_csv('./News Dataset.csv')

In [320]:
df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,category
0,"{'id': None, 'name': 'Yahoo Entertainment'}",Test Yessis,test-setcion,test-setcionThis article originally appeared o...,https://consent.yahoo.com/v2/collectConsent?se...,NaN,2024-05-14T06:19:29Z,click accept partners including part iab trans...,technology
1,"{'id': 'wired', 'name': 'Wired'}",Steven Levy,Don’t Let Mistrust of Tech Companies Blind You...,It’s OK to be doubtful of tech leaders’ grandi...,https://www.wired.com/story/dont-let-mistrust-...,https://media.wired.com/photos/66625436bd6c904...,2024-06-07T13:00:00Z,seems evident almost years first conference ar...,technology
2,"{'id': 'wired', 'name': 'Wired'}",Will Knight,OpenAI Offers a Peek Inside the Guts of ChatGPT,Days after former employees said the company w...,https://www.wired.com/story/openai-offers-a-pe...,https://media.wired.com/photos/6661d484e7f0257...,2024-06-06T17:45:55Z,chatgpt developer openais approach building ar...,technology
3,"{'id': 'wired', 'name': 'Wired'}",Jason Parham,The Rebirth of Queer Cruising Apps,"Looking for casual sex is often seen as taboo,...",https://www.wired.com/story/the-rebirth-of-que...,https://media.wired.com/photos/664fdffb3de340b...,2024-05-27T12:00:00Z,one night past february drinks moody bar light...,technology
4,"{'id': None, 'name': '[Removed]'}",NaN,[Removed],[Removed],https://removed.com,NaN,1970-01-01T00:00:00Z,removed,technology


In [321]:
df.shape

(700, 9)

In [322]:
df.head(1)

,source,author,title,description,url,urlToImage,publishedAt,content,category
0,"{'id': None, 'name': 'Yahoo Entertainment'}",Test Yessis,test-setcion,test-setcionThis article originally appeared o...,https://consent.yahoo.com/v2/collectConsent?se...,NaN,2024-05-14T06:19:29Z,click accept partners including part iab trans...,technology


In [323]:
df.head(1)['description'].values

array(['test-setcionThis article originally appeared on Engadget at https://www.engadget.com/technology/computing/test-setcion-061926526.html?src=rss'],
      dtype=object)

## Missing Data

In [324]:
df.isnull().sum()

source           0
author         120
title            0
description     17
url              0
urlToImage      79
publishedAt      0
content          0
category         0
dtype: int64

In [325]:
df = df.dropna(subset=['urlToImage', 'description'])

In [326]:
df.isnull().sum()

source          0
author         80
title           0
description     0
url             0
urlToImage      0
publishedAt     0
content         0
category        0
dtype: int64

## Duplicated Data

In [327]:
df.duplicated().sum()

0

In [328]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [329]:
df.columns

Index(['source', 'author', 'title', 'description', 'url', 'urlToImage',
       'publishedAt', 'content', 'category'],
      dtype='object')

In [330]:
df.head(1)

,source,author,title,description,url,urlToImage,publishedAt,content,category
1,"{'id': 'wired', 'name': 'Wired'}",Steven Levy,Don’t Let Mistrust of Tech Companies Blind You...,It’s OK to be doubtful of tech leaders’ grandi...,https://www.wired.com/story/dont-let-mistrust-...,https://media.wired.com/photos/66625436bd6c904...,2024-06-07T13:00:00Z,seems evident almost years first conference ar...,technology


In [331]:
type(df['author'][1])

str

In [332]:
def removeSpace(name):
    if pd.isna(name):
        return name
    return ''.join(name.split())

In [333]:
removeSpace(df['author'][1])

'StevenLevy'

In [334]:
df['author'] = df['author'].apply(removeSpace)

In [335]:
df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,category
1,"{'id': 'wired', 'name': 'Wired'}",StevenLevy,Don’t Let Mistrust of Tech Companies Blind You...,It’s OK to be doubtful of tech leaders’ grandi...,https://www.wired.com/story/dont-let-mistrust-...,https://media.wired.com/photos/66625436bd6c904...,2024-06-07T13:00:00Z,seems evident almost years first conference ar...,technology
2,"{'id': 'wired', 'name': 'Wired'}",WillKnight,OpenAI Offers a Peek Inside the Guts of ChatGPT,Days after former employees said the company w...,https://www.wired.com/story/openai-offers-a-pe...,https://media.wired.com/photos/6661d484e7f0257...,2024-06-06T17:45:55Z,chatgpt developer openais approach building ar...,technology
3,"{'id': 'wired', 'name': 'Wired'}",JasonParham,The Rebirth of Queer Cruising Apps,"Looking for casual sex is often seen as taboo,...",https://www.wired.com/story/the-rebirth-of-que...,https://media.wired.com/photos/664fdffb3de340b...,2024-05-27T12:00:00Z,one night past february drinks moody bar light...,technology
5,"{'id': 'wired', 'name': 'Wired'}",GeorgiaGee,Internal Emails Show How a Controversial Gun-D...,NYC mayor Eric Adams wants to test Evolv’s gun...,https://www.wired.com/story/evolv-gun-detectio...,https://media.wired.com/photos/6629392ea620d99...,2024-05-13T16:59:51Z,lot overlap former members nypd adams banks ca...,technology
7,"{'id': None, 'name': 'Gizmodo.com'}",MaxwellZeff,OpenAI Unveils GPT-4 Omni’s Voice Capabilities...,OpenAI unveiled GPT-4 Omni (GPT-4o) during its...,https://gizmodo.com/openai-unveils-gpt-4-omni-...,"https://i.kinja-img.com/image/upload/c_fill,h_...",2024-05-13T18:24:00Z,openai unveiled gpt omni gpt spring update mon...,technology


In [336]:
df['title'] = df['title'].apply(lambda x:x.split())

In [337]:
df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,category
1,"{'id': 'wired', 'name': 'Wired'}",StevenLevy,"[Don’t, Let, Mistrust, of, Tech, Companies, Bl...",It’s OK to be doubtful of tech leaders’ grandi...,https://www.wired.com/story/dont-let-mistrust-...,https://media.wired.com/photos/66625436bd6c904...,2024-06-07T13:00:00Z,seems evident almost years first conference ar...,technology
2,"{'id': 'wired', 'name': 'Wired'}",WillKnight,"[OpenAI, Offers, a, Peek, Inside, the, Guts, o...",Days after former employees said the company w...,https://www.wired.com/story/openai-offers-a-pe...,https://media.wired.com/photos/6661d484e7f0257...,2024-06-06T17:45:55Z,chatgpt developer openais approach building ar...,technology
3,"{'id': 'wired', 'name': 'Wired'}",JasonParham,"[The, Rebirth, of, Queer, Cruising, Apps]","Looking for casual sex is often seen as taboo,...",https://www.wired.com/story/the-rebirth-of-que...,https://media.wired.com/photos/664fdffb3de340b...,2024-05-27T12:00:00Z,one night past february drinks moody bar light...,technology
5,"{'id': 'wired', 'name': 'Wired'}",GeorgiaGee,"[Internal, Emails, Show, How, a, Controversial...",NYC mayor Eric Adams wants to test Evolv’s gun...,https://www.wired.com/story/evolv-gun-detectio...,https://media.wired.com/photos/6629392ea620d99...,2024-05-13T16:59:51Z,lot overlap former members nypd adams banks ca...,technology
7,"{'id': None, 'name': 'Gizmodo.com'}",MaxwellZeff,"[OpenAI, Unveils, GPT-4, Omni’s, Voice, Capabi...",OpenAI unveiled GPT-4 Omni (GPT-4o) during its...,https://gizmodo.com/openai-unveils-gpt-4-omni-...,"https://i.kinja-img.com/image/upload/c_fill,h_...",2024-05-13T18:24:00Z,openai unveiled gpt omni gpt spring update mon...,technology


In [338]:
df['description'] = df['description'].apply(lambda x:x.split())

In [339]:
df['content'] = df['content'].apply(lambda x:x.split())

In [340]:
df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,category
1,"{'id': 'wired', 'name': 'Wired'}",StevenLevy,"[Don’t, Let, Mistrust, of, Tech, Companies, Bl...","[It’s, OK, to, be, doubtful, of, tech, leaders...",https://www.wired.com/story/dont-let-mistrust-...,https://media.wired.com/photos/66625436bd6c904...,2024-06-07T13:00:00Z,"[seems, evident, almost, years, first, confere...",technology
2,"{'id': 'wired', 'name': 'Wired'}",WillKnight,"[OpenAI, Offers, a, Peek, Inside, the, Guts, o...","[Days, after, former, employees, said, the, co...",https://www.wired.com/story/openai-offers-a-pe...,https://media.wired.com/photos/6661d484e7f0257...,2024-06-06T17:45:55Z,"[chatgpt, developer, openais, approach, buildi...",technology
3,"{'id': 'wired', 'name': 'Wired'}",JasonParham,"[The, Rebirth, of, Queer, Cruising, Apps]","[Looking, for, casual, sex, is, often, seen, a...",https://www.wired.com/story/the-rebirth-of-que...,https://media.wired.com/photos/664fdffb3de340b...,2024-05-27T12:00:00Z,"[one, night, past, february, drinks, moody, ba...",technology
5,"{'id': 'wired', 'name': 'Wired'}",GeorgiaGee,"[Internal, Emails, Show, How, a, Controversial...","[NYC, mayor, Eric, Adams, wants, to, test, Evo...",https://www.wired.com/story/evolv-gun-detectio...,https://media.wired.com/photos/6629392ea620d99...,2024-05-13T16:59:51Z,"[lot, overlap, former, members, nypd, adams, b...",technology
7,"{'id': None, 'name': 'Gizmodo.com'}",MaxwellZeff,"[OpenAI, Unveils, GPT-4, Omni’s, Voice, Capabi...","[OpenAI, unveiled, GPT-4, Omni, (GPT-4o), duri...",https://gizmodo.com/openai-unveils-gpt-4-omni-...,"https://i.kinja-img.com/image/upload/c_fill,h_...",2024-05-13T18:24:00Z,"[openai, unveiled, gpt, omni, gpt, spring, upd...",technology


In [341]:
def safe_word_to_list(word):
    L = []
    if isinstance(word, str):
        L.append(word)
        return L

In [342]:
df['author'] = df['author'].apply(safe_word_to_list)

In [343]:
df['category'] = df['category'].apply(safe_word_to_list)
df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,category
1,"{'id': 'wired', 'name': 'Wired'}",[StevenLevy],"[Don’t, Let, Mistrust, of, Tech, Companies, Bl...","[It’s, OK, to, be, doubtful, of, tech, leaders...",https://www.wired.com/story/dont-let-mistrust-...,https://media.wired.com/photos/66625436bd6c904...,2024-06-07T13:00:00Z,"[seems, evident, almost, years, first, confere...",[technology]
2,"{'id': 'wired', 'name': 'Wired'}",[WillKnight],"[OpenAI, Offers, a, Peek, Inside, the, Guts, o...","[Days, after, former, employees, said, the, co...",https://www.wired.com/story/openai-offers-a-pe...,https://media.wired.com/photos/6661d484e7f0257...,2024-06-06T17:45:55Z,"[chatgpt, developer, openais, approach, buildi...",[technology]
3,"{'id': 'wired', 'name': 'Wired'}",[JasonParham],"[The, Rebirth, of, Queer, Cruising, Apps]","[Looking, for, casual, sex, is, often, seen, a...",https://www.wired.com/story/the-rebirth-of-que...,https://media.wired.com/photos/664fdffb3de340b...,2024-05-27T12:00:00Z,"[one, night, past, february, drinks, moody, ba...",[technology]
5,"{'id': 'wired', 'name': 'Wired'}",[GeorgiaGee],"[Internal, Emails, Show, How, a, Controversial...","[NYC, mayor, Eric, Adams, wants, to, test, Evo...",https://www.wired.com/story/evolv-gun-detectio...,https://media.wired.com/photos/6629392ea620d99...,2024-05-13T16:59:51Z,"[lot, overlap, former, members, nypd, adams, b...",[technology]
7,"{'id': None, 'name': 'Gizmodo.com'}",[MaxwellZeff],"[OpenAI, Unveils, GPT-4, Omni’s, Voice, Capabi...","[OpenAI, unveiled, GPT-4, Omni, (GPT-4o), duri...",https://gizmodo.com/openai-unveils-gpt-4-omni-...,"https://i.kinja-img.com/image/upload/c_fill,h_...",2024-05-13T18:24:00Z,"[openai, unveiled, gpt, omni, gpt, spring, upd...",[technology]


In [344]:
type(df['source'][1])

str

In [345]:
df['source'][1]

"{'id': 'wired', 'name': 'Wired'}"

In [346]:
def extract_name(source_str):
    if pd.isna(source_str):
        return None
    
    try:
        source_dict = ast.literal_eval(source_str)  # Safely evaluate the string as a dictionary
        return source_dict.get('name', None)  # Return the value associated with 'name' key
    except ValueError:
        return None

In [347]:
df['source'] = df['source'].apply(extract_name)
df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,category
1,Wired,[StevenLevy],"[Don’t, Let, Mistrust, of, Tech, Companies, Bl...","[It’s, OK, to, be, doubtful, of, tech, leaders...",https://www.wired.com/story/dont-let-mistrust-...,https://media.wired.com/photos/66625436bd6c904...,2024-06-07T13:00:00Z,"[seems, evident, almost, years, first, confere...",[technology]
2,Wired,[WillKnight],"[OpenAI, Offers, a, Peek, Inside, the, Guts, o...","[Days, after, former, employees, said, the, co...",https://www.wired.com/story/openai-offers-a-pe...,https://media.wired.com/photos/6661d484e7f0257...,2024-06-06T17:45:55Z,"[chatgpt, developer, openais, approach, buildi...",[technology]
3,Wired,[JasonParham],"[The, Rebirth, of, Queer, Cruising, Apps]","[Looking, for, casual, sex, is, often, seen, a...",https://www.wired.com/story/the-rebirth-of-que...,https://media.wired.com/photos/664fdffb3de340b...,2024-05-27T12:00:00Z,"[one, night, past, february, drinks, moody, ba...",[technology]
5,Wired,[GeorgiaGee],"[Internal, Emails, Show, How, a, Controversial...","[NYC, mayor, Eric, Adams, wants, to, test, Evo...",https://www.wired.com/story/evolv-gun-detectio...,https://media.wired.com/photos/6629392ea620d99...,2024-05-13T16:59:51Z,"[lot, overlap, former, members, nypd, adams, b...",[technology]
7,Gizmodo.com,[MaxwellZeff],"[OpenAI, Unveils, GPT-4, Omni’s, Voice, Capabi...","[OpenAI, unveiled, GPT-4, Omni, (GPT-4o), duri...",https://gizmodo.com/openai-unveils-gpt-4-omni-...,"https://i.kinja-img.com/image/upload/c_fill,h_...",2024-05-13T18:24:00Z,"[openai, unveiled, gpt, omni, gpt, spring, upd...",[technology]


In [348]:
df['source'] = df['source'].apply(safe_word_to_list)
df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,category
1,[Wired],[StevenLevy],"[Don’t, Let, Mistrust, of, Tech, Companies, Bl...","[It’s, OK, to, be, doubtful, of, tech, leaders...",https://www.wired.com/story/dont-let-mistrust-...,https://media.wired.com/photos/66625436bd6c904...,2024-06-07T13:00:00Z,"[seems, evident, almost, years, first, confere...",[technology]
2,[Wired],[WillKnight],"[OpenAI, Offers, a, Peek, Inside, the, Guts, o...","[Days, after, former, employees, said, the, co...",https://www.wired.com/story/openai-offers-a-pe...,https://media.wired.com/photos/6661d484e7f0257...,2024-06-06T17:45:55Z,"[chatgpt, developer, openais, approach, buildi...",[technology]
3,[Wired],[JasonParham],"[The, Rebirth, of, Queer, Cruising, Apps]","[Looking, for, casual, sex, is, often, seen, a...",https://www.wired.com/story/the-rebirth-of-que...,https://media.wired.com/photos/664fdffb3de340b...,2024-05-27T12:00:00Z,"[one, night, past, february, drinks, moody, ba...",[technology]
5,[Wired],[GeorgiaGee],"[Internal, Emails, Show, How, a, Controversial...","[NYC, mayor, Eric, Adams, wants, to, test, Evo...",https://www.wired.com/story/evolv-gun-detectio...,https://media.wired.com/photos/6629392ea620d99...,2024-05-13T16:59:51Z,"[lot, overlap, former, members, nypd, adams, b...",[technology]
7,[Gizmodo.com],[MaxwellZeff],"[OpenAI, Unveils, GPT-4, Omni’s, Voice, Capabi...","[OpenAI, unveiled, GPT-4, Omni, (GPT-4o), duri...",https://gizmodo.com/openai-unveils-gpt-4-omni-...,"https://i.kinja-img.com/image/upload/c_fill,h_...",2024-05-13T18:24:00Z,"[openai, unveiled, gpt, omni, gpt, spring, upd...",[technology]


In [349]:
df['tags'] = df['source'] + df['author'] + df['title'] + df['description'] + df['content'] + df['category']
df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,category,tags
1,[Wired],[StevenLevy],"[Don’t, Let, Mistrust, of, Tech, Companies, Bl...","[It’s, OK, to, be, doubtful, of, tech, leaders...",https://www.wired.com/story/dont-let-mistrust-...,https://media.wired.com/photos/66625436bd6c904...,2024-06-07T13:00:00Z,"[seems, evident, almost, years, first, confere...",[technology],"[Wired, StevenLevy, Don’t, Let, Mistrust, of, ..."
2,[Wired],[WillKnight],"[OpenAI, Offers, a, Peek, Inside, the, Guts, o...","[Days, after, former, employees, said, the, co...",https://www.wired.com/story/openai-offers-a-pe...,https://media.wired.com/photos/6661d484e7f0257...,2024-06-06T17:45:55Z,"[chatgpt, developer, openais, approach, buildi...",[technology],"[Wired, WillKnight, OpenAI, Offers, a, Peek, I..."
3,[Wired],[JasonParham],"[The, Rebirth, of, Queer, Cruising, Apps]","[Looking, for, casual, sex, is, often, seen, a...",https://www.wired.com/story/the-rebirth-of-que...,https://media.wired.com/photos/664fdffb3de340b...,2024-05-27T12:00:00Z,"[one, night, past, february, drinks, moody, ba...",[technology],"[Wired, JasonParham, The, Rebirth, of, Queer, ..."
5,[Wired],[GeorgiaGee],"[Internal, Emails, Show, How, a, Controversial...","[NYC, mayor, Eric, Adams, wants, to, test, Evo...",https://www.wired.com/story/evolv-gun-detectio...,https://media.wired.com/photos/6629392ea620d99...,2024-05-13T16:59:51Z,"[lot, overlap, former, members, nypd, adams, b...",[technology],"[Wired, GeorgiaGee, Internal, Emails, Show, Ho..."
7,[Gizmodo.com],[MaxwellZeff],"[OpenAI, Unveils, GPT-4, Omni’s, Voice, Capabi...","[OpenAI, unveiled, GPT-4, Omni, (GPT-4o), duri...",https://gizmodo.com/openai-unveils-gpt-4-omni-...,"https://i.kinja-img.com/image/upload/c_fill,h_...",2024-05-13T18:24:00Z,"[openai, unveiled, gpt, omni, gpt, spring, upd...",[technology],"[Gizmodo.com, MaxwellZeff, OpenAI, Unveils, GP..."


In [350]:
dfNew = df.drop(columns=['url','urlToImage','publishedAt','content','description', 'author'])
dfNew.head()

,source,title,category,tags
1,[Wired],"[Don’t, Let, Mistrust, of, Tech, Companies, Bl...",[technology],"[Wired, StevenLevy, Don’t, Let, Mistrust, of, ..."
2,[Wired],"[OpenAI, Offers, a, Peek, Inside, the, Guts, o...",[technology],"[Wired, WillKnight, OpenAI, Offers, a, Peek, I..."
3,[Wired],"[The, Rebirth, of, Queer, Cruising, Apps]",[technology],"[Wired, JasonParham, The, Rebirth, of, Queer, ..."
5,[Wired],"[Internal, Emails, Show, How, a, Controversial...",[technology],"[Wired, GeorgiaGee, Internal, Emails, Show, Ho..."
7,[Gizmodo.com],"[OpenAI, Unveils, GPT-4, Omni’s, Voice, Capabi...",[technology],"[Gizmodo.com, MaxwellZeff, OpenAI, Unveils, GP..."


In [359]:
def preprocess_text(text):
    # Remove non-alphabetic characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z]', ' ', text).lower()
    
    # Tokenize the text
    words = text.split()
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join the words back into a single string
    cleaned_text = ' '.join(words)
    
    return words


In [360]:
type(df['tags'][1])

list

In [361]:
dfNew['tags'] = dfNew['tags'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)

In [362]:
dfNew.head()

,source,title,category,tags
1,[Wired],"[Don’t, Let, Mistrust, of, Tech, Companies, Bl...",[technology],wired stevenlevy don t let mistrust of tech co...
2,[Wired],"[OpenAI, Offers, a, Peek, Inside, the, Guts, o...",[technology],wired willknight openai offers a peek inside t...
3,[Wired],"[The, Rebirth, of, Queer, Cruising, Apps]",[technology],wired jasonparham the rebirth of queer cruisin...
5,[Wired],"[Internal, Emails, Show, How, a, Controversial...",[technology],wired georgiagee internal emails show how a co...
7,[Gizmodo.com],"[OpenAI, Unveils, GPT-4, Omni’s, Voice, Capabi...",[technology],gizmodo com maxwellzeff openai unveils gpt omn...


In [363]:
dfNew['tags'] = dfNew['tags'].apply(preprocess_text)
dfNew.head()

,source,title,category,tags
1,[Wired],"[Don’t, Let, Mistrust, of, Tech, Companies, Bl...",[technology],"[wired, stevenlevy, let, mistrust, tech, compa..."
2,[Wired],"[OpenAI, Offers, a, Peek, Inside, the, Guts, o...",[technology],"[wired, willknight, openai, offer, peek, insid..."
3,[Wired],"[The, Rebirth, of, Queer, Cruising, Apps]",[technology],"[wired, jasonparham, rebirth, queer, cruising,..."
5,[Wired],"[Internal, Emails, Show, How, a, Controversial...",[technology],"[wired, georgiagee, internal, email, show, con..."
7,[Gizmodo.com],"[OpenAI, Unveils, GPT-4, Omni’s, Voice, Capabi...",[technology],"[gizmodo, com, maxwellzeff, openai, unveils, g..."


In [367]:
dfNew['tags'] = dfNew['tags'].apply(lambda x: " ".join(x))

In [368]:
from sklearn.feature_extraction.text import CountVectorizer

In [369]:
cv = CountVectorizer(stop_words='english', max_features=5000)
vectors = cv.fit_transform(dfNew['tags']).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [370]:
vectors.shape

(621, 5000)

In [371]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [372]:
cv.get_feature_names_out()

array(['aarona', 'aaronkatersky', 'aashnagheewalla', ..., 'zhang',
       'zimmerman', 'zoom'], dtype=object)

In [374]:
dfNew.head()['tags'][1]

'wired stevenlevy let mistrust tech company blind power ai ok doubtful tech leader grandiose vision ai future mean technology huge impact seems evident almost year first conference artificial intelligencewhere nascent field leader suggested task would completed within decadethe field char technology'

# Build Neural Network

In [375]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(article_features.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Assuming a binary relevance score
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model

In [ ]:
model.fit(article_features, user_interactions, epochs=10, batch_size=32)

In [ ]:
def recommend_articles(user_profile, top_n=15):
    user_features = vectorizer.transform([preprocess_text(article) for article in user_profile])
    predictions = model.predict(user_features)
    recommended_indices = predictions.argsort()[-top_n:][::-1]
    return [articles[i] for i in recommended_indices]

In [ ]:
# Assuming new_interactions is a list of new user-article interactions
new_article_features = vectorizer.transform([preprocess_text(article) for article in new_articles])
model.fit(new_article_features, new_user_interactions, epochs=5, batch_size=32)